<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Backpropagation-Practice" data-toc-modified-id="Backpropagation-Practice-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Backpropagation Practice</a></span><ul class="toc-item"><li><span><a href="#Data-Science-Unit-4-Sprint-2-Assignment-2" data-toc-modified-id="Data-Science-Unit-4-Sprint-2-Assignment-2-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><em>Data Science Unit 4 Sprint 2 Assignment 2</em></a></span></li><li><span><a href="#Try-building/training-a-more-complex-MLP-on-a-bigger-dataset." data-toc-modified-id="Try-building/training-a-more-complex-MLP-on-a-bigger-dataset.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Try building/training a more complex MLP on a bigger dataset.</a></span></li><li><span><a href="#Stretch-Goals:" data-toc-modified-id="Stretch-Goals:-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Stretch Goals:</a></span></li></ul></li></ul></div>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [2]:
# doing some imports
import numpy as np
import pandas as pd

In [3]:
np.random.seed(42)

# creating the df
df = pd.DataFrame({"x1": [0,0,1,0,1,1,0], "x2": [0,1,0,1,0,1,0], "x3": [1,1,1,0,0,1,0], "y": [0,1,1,1,1,0,0]})

df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [8]:
# Doing normalization
X = df[["x1", "x2", "x3"]]
y = df["y"]

X.shape, y.shape

((7, 3), (7,))

In [16]:
# Creating my class for the multilayered percetron

class Network:

    def __init__(self, inputs=3, hidden=4, output=1):
        # setting up the architecture
        self.inputs = inputs
        self.hidden = hidden
        self.output = output

        # Will initialize the weights randomly
        self.weights1 = np.random.randn(self.inputs, self.hidden)
        self.weights2 = np.random.randn(self.hidden, self.output)

    # doing the sigmoid function 
    def __sigmoid(self, x):
        return 1/(1 + np.exp(-x))

    # doing the feedforward
    def feedforward(self, X):

        # creating the sums for the specific nodes
        self.hidden_sum = np.dot(X, self.weights1)

        self.activated_hidden = self.__sigmoid(self.hidden_sum)
        # now sending to the  output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        # do the acitivation for the output layer
        self.activated_output = self.__sigmoid(self.output_sum)

        return self.activated_output



In [14]:
X.iloc[0]

x1    0
x2    0
x3    1
Name: 0, dtype: int64

In [18]:
# trying a prediction with just the feed forward
net = Network()
output = net.feedforward(X.iloc[0])
print("correct was ", y.iloc[0])
print("guess was ", output)

correct was  0
guess was  [0.14984319]


In [20]:
# Creating my class for the multilayered percetron

class Network:

    def __init__(self, inputs=3, hidden=4, output=1):
        # setting up the architecture
        self.inputs = inputs
        self.hidden = hidden
        self.output = output

        # Will initialize the weights randomly
        self.weights1 = np.random.randn(self.inputs, self.hidden)
        self.weights2 = np.random.randn(self.hidden, self.output)

    # doing the sigmoid function 
    def __sigmoid(self, x):
        return 1/(1 + np.exp(-x))

    # the sigmoid prime or derivitive method
    def __sigmoidPrime(self, x):
        sig = self.__sigmoid(x)
        return sig * (1 - sig)

    # doing the feedforward
    def feedforward(self, X):

        # creating the sums for the specific nodes
        self.hidden_sum = np.dot(X, self.weights1)

        self.activated_hidden = self.__sigmoid(self.hidden_sum)
        # now sending to the  output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        # do the acitivation for the output layer
        self.activated_output = self.__sigmoid(self.output_sum)

        return self.activated_output

    # doing some more methods
    def backprop(self, X, y, o):
        self.o_error = y - o

        # Apply derivative of sigmoid to the  error 
        # to find the amount to adjust
        self.o_delta = self.o_error * self.__sigmoidPrime(self.output_sum)

        self.z2_error = self.o_delta.dot(self.weights2.T)

        # getting the z2 delta
        self.z2_delta = self.z2_error * self.__sigmoidPrime(self.hidden_sum)

        self.weights1 = self.weights1 + X.T.dot(self.z2_delta)
        self.weights2 = self.weights2 + self.activated_hidden.T.dot(self.o_delta)
    
    # run Epoch
    def __run_epoch_W_back_prop(self, X, y):
        output = self.feedforward(X)
        self.backprop(X,y, output)
    # building the function that will do the training for the neural network

    def  train(self, X, y, numEpoch=10000):
        for i in range(numEpoch):
            self.__run_epoch_W_back_prop(X,y)

    def predict(self, x):
        # to do the prediction we wil do feedforward for what is passed 
        # in using the weghts that are now stored
        return self.feedforward(x)

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [ ]:
##### Your Code Here #####

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?